check ffmode else weird syntaxis in the entire text. Example: Acta BotÄ±fmmode\acutea\elseÃ¡\finica VenezuÄ±fmmode\acutee\elseÃ©\filica  
check duplicates and remove them  


Ä±fmmode\\(\w+)\\else.*\\fi

In [1]:
import pandas as pd
import re 
import numpy as np
import os.path

import bibtexparser
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import homogenize_latex_encoding
from bibtexparser.customization import convert_to_unicode

#define parser  method for bibtex
bib_parser = BibTexParser()
bib_parser.customization = convert_to_unicode
# bib_parser.customization = homogenize_latex_encoding

from fuzzywuzzy import fuzz

## Load database

In [2]:
# load abstracts databases (nucleotides and SRA)
df_markers_raw = pd.read_excel("../4_Abstract_extraction_Manual/final_database.xlsx")
df_sra_raw = pd.read_excel("../4_Abstract_extraction_Manual/final_database_SRA.xlsx")

In [3]:
df_raw = pd.concat([df_markers_raw, df_sra_raw])

In [4]:
df_raw.tail()

,N_Index,Old_title,Citation,Google link,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
148,SRA-3906,We included 85 samples of species of Bursera (...,No existe,https://scholar.google.com/scholar?&q=We+inclu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,SRA-3923,Whole-Genome Duplication and Acceleration of M...,"@article{Yu2017Feb,\n author = {Yu, Yan and Xi...",https://scholar.google.com/scholar?&q=Whole-Ge...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,SRA-3931,Widespread adaptive evolution during repeated ...,"@article{Nevado2016Aug,\n author = {Nevado, Br...",https://scholar.google.com/scholar?&q=Widespre...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,SRA-3992,double digest RAD data from Berberis spp. (Ber...,No existe,https://scholar.google.com/scholar?&q=double+d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,SRA-4011,phylogenetic study of Juglans,No existe,https://scholar.google.com/scholar?&q=phylogen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#copy original dataframe (only first three columns)
df = df_raw.iloc[:, 0:3].copy()

#remove rows that has no bibtex citation
df = df[df["Citation"].str.startswith("@")]

In [6]:
#create final df, in which tematic columns will be added.
#in this dataframe will be the information for every citation field

final_df = pd.DataFrame()


for row in df.itertuples():
    new_row_values = {}
# for row in df.head(5).itertuples():
    #parse bibtex
    citation = bibtexparser.loads(row.Citation, bib_parser)
    citation = citation.entries[-1]
    
    #add N_Index 
    # row_values["N_Index"] = row.N_Index

    #add columns for each citation field (added prefix c. = citation)
    for field in citation:
        # #verify weird syntax for special chars 
        # citation[field] = re.sub(r"Ä±fmmode\\(\w+)\\else.*\\fi", r"\1", citation[field])


        new_row_values[f"c.{field.lower()}"] = citation[field]

    #count names and add them to the final df
    ##split names by " and "
    try:
        names = citation["author"].split(" and ")
        authors = bibtexparser.customization.getnames(names)
        n_authors = len(authors)
    except:
        authors = "ERROR"
        n_authors = "ERROR"
    
    #replace names with a organized structure from bibtexparser
    new_row_values["c.author"] = authors
    new_row_values["s.n_authors"] = str(n_authors) #in this prefix s. is added to refer statistics

    #add old title
    new_row_values["a.oldtitle"] = row.Old_title #prefix a. is for archive
    
    #compare old and new title
    try:
        titles_similitud = fuzz.token_sort_ratio(row.Old_title.lower(), citation["title"].lower())
        new_row_values["v.simil_titles"] = titles_similitud #prefix v. is for verification
    except:
        new_row_values["v.simil_titles"] = 0

    row_to_add = pd.Series(new_row_values, name=row.N_Index)
    final_df = final_df.append(row_to_add, sort=True)

In [7]:
final_df

,a.oldtitle,c.abstract,c.address,c.author,c.booktitle,c.doi,c.entrytype,c.eprint,c.id,c.isbn,...,c.note,c.number,c.pages,c.publisher,c.title,c.url,c.volume,c.year,s.n_authors,v.simil_titles
0,Andean-centred' genera in the short-branch cla...,Aim We test biogeographical hypotheses regardi...,NaN,"[Pirie, Michael D., Chatrou, Lars W., Mols, Jo...",NaN,10.1111/j.1365-2699.2005.01388.x,article,NaN,Pirie2006Jan,NaN,...,NaN,1,31--46,"John Wiley & Sons, Ltd",`Andean-centred′ genera in the short-branch cl...,NaN,33,2006,5,100.0
23,A 10-gene phylogeny of Solanum section Herpyst...,Premise of the study: Solanum section Herpysti...,NaN,"[Tepe, Eric J., Farruggia, Frank T., Bohs, Lynn]",NaN,10.3732/ajb.1000516,article,NaN,Tepe2011Aug,NaN,...,NaN,8,1356--1365,"John Wiley & Sons, Ltd",A 10-gene phylogeny of Solanum section Herpyst...,NaN,98,2011,3,100.0
26,A 250 plastome phylogeny of the grass family (...,The systematics of grasses has advanced throug...,NaN,"[Saarela, Jeffery M., Burke, Sean V., Wysocki,...",NaN,10.7717/peerj.4299,article,NaN,Saarela2018Feb,NaN,...,NaN,NaN,e4299,PeerJ Inc.,A 250 plastome phylogeny of the grass family (...,NaN,6,2018,10,100.0
29,A 313 plastome phylogenomic analysis of Pooide...,"In this study, we analyzed 313 plastid genomes...",NaN,"[Orton, Lauren M., Barberıfmmode\acutea\elseá\...",NaN,10.1016/j.ympev.2021.107110,article,NaN,Orton2021Jun,NaN,...,NaN,NaN,107110,Academic Press,A 313 plastome phylogenomic analysis of Pooide...,NaN,159,2021,7,100.0
32,A Chloroplast Phylogeny of Agavaceae subfamily...,Abstractıfmmode---\else—\fi Agavaceae subfamil...,NaN,"[Halpin, Kate M., Fishbein, Mark]",NaN,10.1600/036364413X674850,article,NaN,Halpin2013Dec,NaN,...,NaN,4,996--1011,American Society of Plant Taxonomists,A Chloroplast Phylogeny of Agavaceae subfamily...,NaN,38,2013,2,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRA-3855,Using transcriptomes to explore the evolution ...,Caryophyllales are a highly diverse and large ...,NaN,"[Pucker, Boas, Feng, Tao, Brockington, Samuel F.]",NaN,NaN,article,646133,Pucker2020Feb,NaN,...,NaN,NaN,646133,Cold Spring Harbor Laboratory,Next generation sequencing to investigate geno...,https://doi.org/10.1101/646133,NaN,2020,3,50.0
SRA-3870,Viburnum lentago clade RADseq data,Premise We take an integrative approach in ass...,NaN,"[Spriggs, Elizabeth L., Schlutius, Caroline, E...",NaN,10.1002/ajb2.1292,article,NaN,Spriggs2019Jun,NaN,...,NaN,6,833--849,"John Wiley & Sons, Ltd",Differences in flowering time maintain species...,NaN,106,2019,7,43.0
SRA-3871,Viburnum nudum complex RADseq data,Species are the starting point for most studie...,NaN,"[Spriggs, Elizabeth L., Eaton, Deren A. R., Sw...",NaN,10.1093/sysbio/syy084,article,NaN,Spriggs2019Mar,NaN,...,NaN,2,187--203,Oxford Academic,Restriction-Site-Associated DNA Sequencing Rev...,NaN,68,2019,6,34.0
SRA-3923,Whole-Genome Duplication and Acceleration of M...,The pattern and rate of genome evolution have ...,NaN,"[Yu, Yan, Xiang, Qiuyun, Manos, Paul S., Solti...",NaN,10.1371/journal.pone.0171361,article,NaN,Yu2017Feb,NaN,...,NaN,2,e0171361,Public Library of Science,Whole-genome duplication and molecular evoluti...,NaN,12,2017,9,71.0


In [11]:
#remove duplicates
final_df = final_df.drop_duplicates(subset="c.title", keep="first")

In [12]:
# export database as csv
# in this database tematic extraction will be done
final_df.to_csv("database_for_tematic_extraction.csv")